In [1]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.colors import LogNorm
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

In [2]:
data = np.load('us_topo.npy')
X, y = data[:,:-1], data[:,-1]

## Some support functions

In [3]:
def print_GPs(GPs, name):
    plt.plot(X_, 0.5*np.sin(3*X_), label='ground truth')
    for i, gp in enumerate(GPs):
        print('gp - '+name+' (',i+1,'): ', gp, '\nkernel:', gp.kernel_)
        print('theta:', gp.kernel_.theta, np.exp(gp.kernel_.theta))
        print('likelihood:', gp.log_marginal_likelihood_value_)


    for i, gp in enumerate(GPs):
        y_mean, y_cov = gp.predict(X_[:, np.newaxis], return_cov=True)
        #plt.plot(X_, y_mean, 'k', lw=3, zorder=9, label='sklearn kernel: '+str(i+1))
        plt.plot(X_, y_mean, label='sklearn kernel: '+str(i+1))

    plt.legend()
    plt.tight_layout()

In [4]:
def make_plots(GPs):
    fig, ax = plt.subplots(2,1, figsize=(12,10))
    #plt.title('Log Marginal Likelihood Surfaces with Prediction')
    for i, gp in enumerate(GPs):
        ax = plt.subplot(2,1,i+1)
        #ax = plt.subplot(1,1,1)

        plt.scatter(np.exp(gp.kernel_.theta[1]), np.exp(gp.kernel_.theta[2]), c='red', marker='x')

        a, b = gp.kernel_.bounds[1:]
        theta0 = np.linspace(a[0], a[1], 49)
        theta1 = np.linspace(b[0], b[1], 50)
        Theta0, Theta1 = np.meshgrid(theta0, theta1)
        LML = np.empty(Theta0.shape)
        for i in range(Theta0.shape[0]):
            for j in range(Theta0.shape[1]):
                LML[i,j] = gp.log_marginal_likelihood(np.array([gp.kernel_.theta[0], Theta0[i, j], Theta1[i, j]]))

        #LML = np.array(LML).T
        vmin, vmax = (-LML).min(), (-LML).max()
        level = np.around(np.logspace(np.log10(vmin), np.log10(vmax), 50), decimals=1)
        plt.contour(np.exp(Theta0), np.exp(Theta1), -LML,
                    levels=level, colors='black', linewidths=1., norm=LogNorm(vmin=vmin, vmax=vmax))
        plt.yscale('log')
        plt.xscale('log')
        subAx = inset_axes(ax,
                        width="70%", # width = 30% of parent_bbox
                        height="60%", # height : 1 inch
                        loc='lower right')
        subAx.plot(X_, 0.5*np.sin(3*X_), color='black', label='ground truth')
        y_mean, y_cov = gp.predict(X_[:, np.newaxis], return_cov=True)
        subAx.plot(X_, y_mean, label='prediction')

        subAx.scatter(X, y, color='black', marker='o')
        subAx.fill_between(X_, y_mean - np.sqrt(np.diag(y_cov)),
                     y_mean + np.sqrt(np.diag(y_cov)),
                     alpha=0.5, color='grey')
        subAx.set_xticks([])
        subAx.set_xticks([], minor=True)
        subAx.set_yticks([])
        subAx.set_yticks([], minor=True)

        subAx.legend()
    #plt.show()
    plt.savefig('hgdl_plot')

## Run HGDL

Overwrite the normal gaussian process regressor to be the one that I define

In [5]:
from fit import fit
GaussianProcessRegressor.fit = fit

In [ ]:
hgdl_GPs = GaussianProcessRegressor(kernel=1.0*RBF(1.0), optimizer='hgdl', random_state=42).fit(
    X, y, num_individuals=3, num_epochs=5, max_local=2, r=3.0)

In [ ]:
print_GPs(hgdl_GPs, 'hgdl')

In [ ]:
del hgdl_GPs[1]

In [ ]:
make_plots(hgdl_GPs)

In [ ]:
import numpy as np
a = np.genfromtxt("SRTM_RAMP2_TOPO_2000-02-11_rgb_3600x1800.CSV", delimiter = ",")
print(a.shape)
i = np.where(a == 99999.)
a[i] = 0.0
plt.imshow(a);plt.show()

map1 = a[420:620, 550:1050]
map1 = map1[::2,::2]   
plt.imshow(map1);plt.show()

data = np.empty((100*250,3))
counter = 0

for i in range(100):
    for j in range(250):
        data[counter,0] = i
        data[counter,1] = j
        data[counter,2] = map1[i,j]
        counter += 1

np.save("us_topo",data)

